In [ ]:
import os
import pydicom
import pandas as pd

main_folder = "/home/omen/Documents/Nafisha/AIIMsVSdata/DICOM_GammaKnife"
records = []

for root, dirs, files in os.walk(main_folder):
    dicom_files = [file for file in files if file.lower().endswith(".dcm")]
    
    if dicom_files:
        dcm_path = os.path.join(root, dicom_files[0])
        try:
            dcm = pydicom.dcmread(dcm_path, stop_before_pixels=True)
            record = {
                "PatientID": getattr(dcm, "PatientID", ""),
                "PatientsName": str(getattr(dcm, "PatientName", "")).replace("^", " "),
                "StudyInstanceUID": getattr(dcm, "StudyInstanceUID", ""),
                "StudyDate": getattr(dcm, "StudyDate", ""),
                "StudyDescription": getattr(dcm, "StudyDescription", ""),
                "Modality": getattr(dcm, "Modality", ""),
                "SeriesDescription": getattr(dcm, "SeriesDescription", ""),
                "StudyFolderPath": root
            }
            records.append(record)
        except Exception as e:
            print(f"Skipping file: {dcm_path} due to error: {e}")

# Create DataFrame
df = pd.DataFrame(records)

In [ ]:
import numpy as np
len(np.unique(df['PatientID']))

In [ ]:
data_MR = df[df['Modality']=='MR']
data_MR

#### Special for Random Forest

In [ ]:
# Step 1: Count number of unique StudyDates per PatientID
studydate_counts = data_MR.groupby('PatientID')['StudyDate'].nunique()

# Step 2: Identify PatientIDs that have >1 unique StudyDate
valid_patients = studydate_counts[studydate_counts > 1].index

# Step 3: Filter the original DataFrame to include only those patients
filtered_data_MR = data_MR[data_MR['PatientID'].isin(valid_patients)]

filtered_data_MR

In [ ]:
# Step 1: Group by PatientID and count unique StudyDescriptions
study_counts = filtered_data_MR.groupby("PatientID")["StudyDescription"].nunique()

# Step 2: Keep only those PatientIDs with more than one unique StudyDescription
valid_patients = study_counts[study_counts > 1].index

# Step 3: Filter the original DataFrame
filtered_result = filtered_data_MR[filtered_data_MR["PatientID"].isin(valid_patients)]

Below cell is to make a copy of data used for Random Forest training so that while creating a mask, original data does not get changed

In [ ]:
import os
import shutil

new_dir = '/home/omen/Documents/Nafisha/RandomForestData/images'

for idx, row in filtered_result.iterrows():
    image_folder_path = row['StudyFolderPath']  
    base_folder = os.path.basename(image_folder_path)
    dest_folder = os.path.join(new_dir, base_folder)

    os.makedirs(dest_folder, exist_ok=True)

    # Copy contents of image_folder_path into dest_folder
    for item in os.listdir(image_folder_path):
        s = os.path.join(image_folder_path, item)
        d = os.path.join(dest_folder, item)
        if os.path.isdir(s):
            shutil.copytree(s, d, dirs_exist_ok=True)
        else:
            shutil.copy2(s, d)  # copy2 to preserve metadata

    print(f"Copied: {image_folder_path} -> {dest_folder}")

    filtered_result.at[idx, "StudyFolderPath"] = dest_folder


creating different folder for RTSS,RTPLAN and RTDOSE

In [ ]:
import os
import shutil

# Destination root folder
rtfiles = "/home/omen/Documents/Nafisha/RandomForestData/rtfiles"

# Filenames to move
rt_filenames = ['RTDOSE.dcm', 'RTPLAN.dcm', 'RTSS.dcm']

for idx, row in filtered_result.iterrows():
    study_path = row['StudyFolderPath']
    if not os.path.exists(study_path):
        print(f"Study folder does not exist: {study_path}")
        continue

    # Folder name to create under rtfiles
    folder_name = os.path.basename(study_path)
    destination_folder = os.path.join(rtfiles, folder_name)
    os.makedirs(destination_folder, exist_ok=True)

    for rt_file in rt_filenames:
        source_path = os.path.join(study_path, rt_file)
        dest_path = os.path.join(destination_folder, rt_file)

        if os.path.exists(source_path):
            shutil.move(source_path, dest_path)
            print(f"Moved: {source_path} -> {dest_path}")
        else:
            print(f"File not found: {source_path}")
            
    filtered_result.at[idx, "rtfiles"] = destination_folder

Generating Maks

In [ ]:
from platipy.dicom.io.rtstruct_to_nifti import convert_rtstruct

In [ ]:
maskfolder= "/home/omen/Documents/Nafisha/RandomForestData/masks"

for idx, row in filtered_result.iterrows():
    image_folder_path = row['StudyFolderPath']
    base_folder = os.path.basename(image_folder_path)
    mask_folder_path = os.path.join(maskfolder, base_folder)
    rtss_path = os.path.join(row['rtfiles'], 'RTSS.dcm')

    # print(os.path.exists(rtss_path))
    
    os.makedirs(mask_folder_path, exist_ok=True)
    
    convert_rtstruct(
        dcm_img= image_folder_path,
        dcm_rt_file = rtss_path,
        output_dir = mask_folder_path
    )
    
    filtered_result.at[idx, "mask_folder_path"] = mask_folder_path

In [ ]:
outliers = ['100213278', '100480978']

filtered_result = filtered_result[~filtered_result['PatientID'].isin(outliers)]
filtered_result

In [ ]:
filtered_result["mask_path"] = ""

for idx, row in filtered_result.iterrows():
    folder = row['mask_folder_path']

    if not os.path.exists(folder):
        print(f"Folder does not exist: {folder}")
        continue

    # List all files in the folder
    for file in os.listdir(folder):
        if "tumor" in file.lower() and (file.endswith(".nii") or file.endswith(".nii.gz")):
            full_path = os.path.join(folder, file)
            print(full_path)
            filtered_result.loc[idx, "mask_path"] = full_path
            break  # stop after the first match

    if filtered_result.loc[idx, "mask_path"] == "":
        print(f"No tumor mask found in: {folder}")

In [ ]:
import SimpleITK as sitk
reader = sitk.ImageSeriesReader()
sitk.ProcessObject_SetGlobalWarningDisplay(False)

In [ ]:
import numpy as np

def prefer_t1(group):
    # Check if any SeriesDescription contains "t1" (case-insensitive)
    mask_t1 = group['SeriesDescription'].str.contains('t1', case=False, na=False)
    if mask_t1.any():
        # Return only rows with t1 in SeriesDescription
        return group[mask_t1].iloc[[0]]
    else:
        # No t1 found, keep the first row
        return group.iloc[[0]]

new_filtered_result = filtered_result.groupby(['PatientID', 'StudyDate'], group_keys=False).apply(prefer_t1).reset_index(drop=True)

# Radiomics Feature Extraction

In [ ]:
from radiomics import featureextractor

features_dict = {}
x = []

for i in range(len(weight_folders)):
    segmentation_path = seg_paths[i]
    segmentation_image = sitk.ReadImage(segmentation_path)

    reader = sitk.ImageSeriesReader()
    dicom_names = reader.GetGDCMSeriesFileNames(weight_folders[i])
    reader.SetFileNames(dicom_names)
    image = reader.Execute()

    patient_id = patient_ids[i]
    patient_year = study_year[i]
        
    # print(patient_id,patient_year)
    # print(image.GetSize(), segmentation_image.GetSize())
    segmentation_resampled = resample_segmentation_to_image(segmentation_image, image)
    segmentation_resampled_array = sitk.GetArrayFromImage(segmentation_resampled)
    
    if np.sum(segmentation_resampled_array) == 0:
        print(f"Warning: The resampled segmentation mask for {segmentation_path} is empty (no segmented regions found).")
        continue

    extractor = featureextractor.RadiomicsFeatureExtractor()
    extractor.enableAllFeatures()

    features = extractor.execute(image, segmentation_resampled)
    
    x.append(f"{patient_id}_{patient_year}")

    # Store features in a list to handle repeated patient IDs
    key = f"{patient_id}_{patient_year}"
    if key in features_dict:
        suffix = 1
        while f"{key}_{suffix}" in features_dict:
            suffix += 1
        key = f"{key}_{suffix}"

    if key not in features_dict:
        features_dict[key] = []
    features_dict[key].append(features)

In [ ]:
voxel_volume_feature = {}
for key, feature_list in features_dict.items():
    for feature in feature_list:
        value = feature.get('original_shape_MeshVolume', None)
        voxel_volume_feature[key] = value.item() if isinstance(value, np.ndarray) else value
len(voxel_volume_feature)

In [ ]:
import pandas as pd
import numpy as np

data = []
for key, value in voxel_volume_feature.items():
    parts = key.split('_')

    if len(parts) > 2:
        patient_id = parts[0]
        year = parts[1] + "_" + parts[2]  # Create year_1 format
    else:
        patient_id, year = parts[0], parts[1]

    data.append((patient_id, year, value))


df = pd.DataFrame(data, columns=['Patient_ID', 'Year', 'Volume'])

# Pivot the DataFrame to create a table with Patient_ID as the index, Year as columns, and Volume as the values
volume_table = df.pivot(index='Patient_ID', columns='Year', values='Volume')

# Replace missing values with "N/A"
volume_table = volume_table.fillna("NaN")

In [ ]:
import pandas as pd
import numpy as np

volume_diff_column = []


for idx, row in volume_table.iterrows():

    numeric_values = pd.to_numeric(row.dropna(), errors='coerce').dropna().values
    if len(numeric_values) > 1:
        first_numeric = int(numeric_values[0])
        last_numeric = int(numeric_values[-1])
        volume_diff = last_numeric - first_numeric
    else:
        volume_diff = np.nan


    volume_diff_column.append(volume_diff)

# Add the calculated 'volume_diff' as a new column
volume_table['volume_diff'] = volume_diff_column

# Create a new column based on whether the volume_diff is positive or negative
volume_table['change'] = volume_table['volume_diff'].apply(lambda x: 'increased' if x > 0 else 'not increased')

In [ ]:
feature_names = ['original_shape_MajorAxisLength',
 'original_shape_Maximum2DDiameterRow',
 'original_shape_Maximum3DDiameter',
 'original_firstorder_Kurtosis',
 'original_glcm_MCC',
 'original_gldm_GrayLevelNonUniformity',
 'original_glrlm_GrayLevelNonUniformity',
 'original_glrlm_RunLengthNonUniformity',
 'original_ngtdm_Busyness',
 'original_ngtdm_Coarseness']
len(feature_names)

In [ ]:
for feature in feature_names:
    volume_table[feature] = float('nan')

In [ ]:
for idx, row in volume_table.iterrows():
    numeric_values = pd.to_numeric(row.dropna(), errors='coerce').dropna()
    if numeric_values.empty:
        continue

    year = min(numeric_values.index.tolist())  # earliest available year
    dict_key = f"{idx}_{year}"

    # Check if the key exists in features_dict
    if dict_key in features_dict:
        feature_values = features_dict[dict_key][0]
        for feature in feature_names:
            volume_table.at[idx, feature] = feature_values[feature]
    else:
        print(f"Warning: {dict_key} not found in features_dict.")

In [ ]:
volume_table = volume_table.loc[:, ~volume_table.columns.str.match(r'^\d+$')]

In [ ]:
volume_table.drop(columns=['volume_diff'], inplace=True)
volume_table

In [ ]:
volume_table['change'] = volume_table['change'].map({'increased': 1, 'not increased': 0})

In [ ]:
file_path = '/home/omen/Documents/Nafisha/RandomForestData/radiomicsFeat.csv'
volume_table.to_csv(file_path)